In [ ]:
import glob
import os
import sys
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
import supervision as sv
from ultralytics import YOLO
import time
import pygame

try:
    sys.path.append(glob.glob('../../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

import carla

In [ ]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

client.load_world('Town02')

world = client.get_world()
spectator = world.get_spectator()

## Some helpful functions used in this notebook

In [ ]:
IMAGE_SIZE = 640

def move_spectator_to(transform, distance=5.0, x=0, y=0, z=4, yaw=0, pitch=-30, roll=0):
    back_location = transform.location - transform.get_forward_vector() * distance
    
    back_location.x += x
    back_location.y += y
    back_location.z += z
    transform.rotation.yaw += yaw
    transform.rotation.pitch = pitch
    transform.rotation.roll = roll
    
    spectator_transform = carla.Transform(back_location, transform.rotation)
    
    spectator.set_transform(spectator_transform)

def spawn_vehicle(vehicle_index=0, spawn_index=0, pattern='vehicle.*'):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

def draw_on_screen(world, transform, content='O', color=carla.Color(0, 255, 0), life_time=20):
    world.debug.draw_string(transform.location, content, color=color, life_time=life_time)

def spawn_camera(attach_to=None, transform=carla.Transform(carla.Location(x=0.7, z=1.8), carla.Rotation(pitch=5, yaw=35)), width=IMAGE_SIZE, height=IMAGE_SIZE, foV=50):
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(width))
    camera_bp.set_attribute('image_size_y', str(height))
    camera_bp.set_attribute('fov', str(foV))
    camera_bp.set_attribute('sensor_tick', '0')
    camera = world.spawn_actor(camera_bp, transform, attach_to=attach_to)
    return camera

## Camera sensor

Let's spawn a vehicle and a camera in a particular position and see what the camera sees.

How many images have been captured by the camera so far?

### Live camera feed

We can also visualize the camera feed in real-time. This is useful for debugging and understanding what the camera sees.

We can use the `opencv` library to create a window and display the camera feed.

In [ ]:
#!pip install roboflow
# from roboflow import Roboflow
# rf = Roboflow(api_key="...")
# project = rf.workspace("wawan-pradana").project("cinta_v2")
# dataset = project.version(1).download("yolov8")

# model = YOLO('yolov8n.pt')
# results = model.train(data="data.yaml", epochs=100, imgsz=640, save=True, device="cpu")


In [ ]:
from ultralytics import YOLO

IMAGE_SIZE_SPEED = 416
# Carica il modello con i pesi migliori
model = YOLO("runsSpeed/detect/train/weights/best.pt")
modelTrafficLight = YOLO("runsTrafficLight/detect/train3/weights/best.pt")


In [ ]:
# all_actors = world.get_actors()
# traffic_lights = all_actors.filter('traffic.traffic_light')

# for light in traffic_lights:
#     # Imposta il semaforo su verde
#     light.set_state(carla.TrafficLightState.Green)
#     light.set_green_time(366666.0)

weather = world.get_weather()

# Imposta il cielo notturno
# weather.sun_azimuth_angle = 0  # Angolo dell'azimut del sole
# weather.sun_altitude_angle = -90  # Angolo di altezza del sole per simulare la notte
# weather.cloudiness = 100  # Copertura nuvolosa massima
# weather.precipitation = 100  # Pioggia massima
# weather.precipitation_deposits = 100  # Depositi di pioggia sul terreno
# weather.wind_intensity = 100  # Vento al massimo
# weather.fog_density = 100  # Un po' di nebbia per aggiungere realismo
# weather.fog_distance = 1  # Distanza di visibilità ridotta

weather.sun_azimuth_angle = 0  # Angolo dell'azimut del sole
weather.sun_altitude_angle = 90  # Angolo di altezza del sole per simulare la notte
weather.cloudiness = 0  # Copertura nuvolosa massima
weather.precipitation = 0  # Pioggia massima
weather.precipitation_deposits = 0  # Depositi di pioggia sul terreno
weather.wind_intensity = 0  # Vento al massimo
weather.fog_density = 30  # Un po' di nebbia per aggiungere realismo

# Applica le impostazioni
world.set_weather(weather)
    

In [ ]:
def colorSpeedLimit(current_speed, speed_limit):
    if speed_limit == None or int(current_speed) <= int(speed_limit):
        return (0, 255, 0) 
    else:
        return (0, 0, 255)

In [ ]:
def calcColor():
    if traffic_light == "red":
        return (0, 0, 255)
    elif traffic_light == "yellow":
        return (0, 128, 255)
    elif traffic_light == "green":
        return (0, 255, 0)
    else:
        return (255, 255, 255)

In [ ]:
def camera_callback(image):
    global last_analysis_time, last_speed_limit, speed_car
    # Ottieni il timestamp attuale
    current_time = time.time()

    velocity_car = vehicle.get_velocity()
    speed_car = 3.6 * (velocity_car.x**2 + velocity_car.y**2 + velocity_car.z**2)**0.5  # Conversione da m/s a km/h

    # Converti l'immagine raw in un array NumPy
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    # Assicurati che l'immagine abbia il formato BGRA (4 canali)
    image_np = array.reshape((IMAGE_SIZE, IMAGE_SIZE, 4))  # Immagine BGRA
    image_bgr = image_np[:, :, :3]  
    resized_image = cv2.resize(image_bgr, (IMAGE_SIZE_SPEED, IMAGE_SIZE_SPEED), interpolation=cv2.INTER_AREA)

    # Verifica se è passato almeno 1 secondo dall'ultima analisi
    if current_time - last_analysis_time >= analysis_interval:
        last_analysis_time = current_time  # Aggiorna il timestamp

        # Rileva gli oggetti nell'immagine
        results = model(resized_image)
        #resultsTrafficLight = modelTrafficLight(image_bgr)

        limit_id = None
        confidence_max = 0

            # Class IDs
        class_ids = results[0].boxes.cls.numpy()
        confidences = results[0].boxes.conf.numpy()
        
        for class_id, confidence in zip(class_ids, confidences):
            if confidence > confidence_max:
                limit_id = class_id
                confidence_max = confidence

        if confidence_max > 0.87:
            print(confidence_max)
            class_name = class_names[int(limit_id)]  # Ottieni il nome della classe
            try:
                last_speed_limit = class_name.split(" ")[2]  # Usa confidenza come esempio
            except:
                pass

In [ ]:
def camera_view_callback(image):
    global video_output
    # Converti l'immagine raw in un array NumPy
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    # Assicurati che l'immagine abbia il formato BGRA (4 canali)
    image_np = array.reshape((IMAGE_SIZE, IMAGE_SIZE, 4))  # Immagine BGRA
    video_output = image_np  # Mantieni il frame per visualizzazione

In [ ]:
def camera_traffic_callback(image):
    global last_analysis_time_trafficLight, traffic_light, red_over
    # Ottieni il timestamp attuale
    current_time = time.time()

    # Converti l'immagine raw in un array NumPy
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    # Assicurati che l'immagine abbia il formato BGRA (4 canali)
    image_np = array.reshape((IMAGE_SIZE, IMAGE_SIZE, 4))  # Immagine BGRA
    image_bgr = image_np[:, :, :3].copy()  # Crea una copia modificabile dell'immagine BGR
    # Determina la frazione dell'immagine che vuoi oscurare
    fraction_to_black = 0.1  # Per esempio, il 20% inferiore dell'immagine
    height = image_bgr.shape[0]
    black_start_row = int(height * (1 - fraction_to_black))
    image_bgr[black_start_row:, :] = 0  # Imposta a nero (0, 0, 0)
    image_bgr[:, 500:] = 0
    #resized_image = cv2.resize(image_bgr, (IMAGE_SIZE_SPEED, IMAGE_SIZE_SPEED), interpolation=cv2.INTER_AREA)

    accentuated_image = image_bgr.copy()

    # Aumenta il canale Rosso
    accentuated_image[:, :, 2] = cv2.add(accentuated_image[:, :, 2], 15)  # Aumenta Rosso (+50)

    # Riduci il canale Verde
    accentuated_image[:, :, 1] = cv2.subtract(accentuated_image[:, :, 1], 0)  # Riduci Verde (-30)

    # Riduci il canale Blu
    accentuated_image[:, :, 0] = cv2.subtract(accentuated_image[:, :, 0], 20)  # Riduci Blu (-30)

    # Assicurati che i valori restino nel range [0, 255]
    resized_image = np.clip(accentuated_image, 0, 255).astype(np.uint8)
    video_output = resized_image

    # Verifica se è passato almeno 1 secondo dall'ultima analisi
    if current_time - last_analysis_time_trafficLight >= analysis_interval_trafficLight:
        if int(speed_car) == 0:
            red_over = False
        last_analysis_time_trafficLight = current_time  # Aggiorna il timestamp

        # Rileva gli oggetti nell'immagine
        results = modelTrafficLight(resized_image)

        trafficLight_id = None
        confidence_max = 0

        # Class IDs
        class_ids = results[0].boxes.cls.numpy()
        confidences = results[0].boxes.conf.numpy()

        if class_ids.size == 0:
            traffic_light = "Non individuato"
        else:
            for class_id, confidence in zip(class_ids, confidences):
                if confidence > confidence_max:
                    trafficLight_id = class_id
                    confidence_max = confidence

            class_name = class_names_trafficLight[int(trafficLight_id)]
            if confidence_max > 0.50 and class_name=="red" or confidence_max > 0.42 and class_name!="red":
                traffic_light = class_name
                print(confidence_max)

                if class_name == "green":
                    red_over = False

                x_min, y_min, x_max, y_max = results[0].boxes[0].xyxy[0].numpy()  # Estrai (x_min, y_min, x_max, y_max)

                # Calcola il centro del box
                box_center_x = (x_min + x_max) / 2

                # Verifica se è all'estrema destra (ad esempio, nel 20% dell'immagine)
                if box_center_x > IMAGE_SIZE_SPEED * 0.75 and class_name == 'red':
                    print(f"Semaforo rosso all'estrema destra: Box ({x_min}, {y_min}, {x_max}, {y_max})")
                    if(int(speed_car) <= 50): # Freniamo solo se andiamo sotto ai 50km/h
                        red_over = True

            # Cerco dove ha individuato il semaforo e se lo sorpasso freno:
            # Verifica se ci sono bounding boxes rilevati

            # Coordinate del bounding box
            


In [ ]:
display = pygame.display.set_mode((50, 50))
pygame.display.set_caption("Controllo Manuale CARLA")

# Velocità di sterzo e accelerazione
steer_increment = 0.02
throttle_increment = 0.1
steer = 0.0
throttle = 0.0
brake = 0.0
gear = "Drive"  # Stato iniziale (marcia avanti)
speed_control_activate = False
red_over = False

# Costanti del controllore proporzionale
KP_THROTTLE = 0.15  # Guadagno proporzionale per accelerazione
KP_BRAKE = 0.02    # Guadagno proporzionale per frenata
DEAD_ZONE = 3.0    # Zona morta attorno al limite di velocità
MIN_THROTTLE = 0.2  # Soglia minima di accelerazione
MIN_BRAKE = 0.1     # Soglia minima di frenata

vehicle = spawn_vehicle()
camera = spawn_camera(attach_to=vehicle)
camera_view = spawn_camera(attach_to=vehicle, transform=carla.Transform(carla.Location(x=1, z=1.5), carla.Rotation(pitch=0, yaw=0)), width=IMAGE_SIZE, height=IMAGE_SIZE, foV=90)
camera_trafficLight = spawn_camera(attach_to=vehicle, transform=carla.Transform(carla.Location(x=0, y=1 , z=1.2), carla.Rotation(pitch=20, yaw=20)), width=IMAGE_SIZE, height=IMAGE_SIZE, foV=40)

# Inizializza il frame di output
video_output = np.zeros((IMAGE_SIZE, IMAGE_SIZE, 3), dtype=np.uint8)

# Variabile per tenere traccia del tempo dell'ultima analisi
last_analysis_time = 0  # Inizializza l'ultima analisi a 0 secondi
last_analysis_time_trafficLight = 0
analysis_interval = 0.3  # Intervallo in secondi tra le analisi
analysis_interval_trafficLight = 0.1

# Variabile per tenere traccia dell'ultimo limite di velocità
last_speed_limit = None
speed_car = 0
traffic_light = ""

class_names = model.names 
class_names_trafficLight = modelTrafficLight.names

# Assegna il callback alla telecamera
camera.listen(lambda image: camera_callback(image))

camera_view.listen(lambda image: camera_view_callback(image))

camera_trafficLight.listen(lambda image: camera_traffic_callback(image))

# Mostra i risultati
cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)

try:
    clock = pygame.time.Clock()
    event_timer = 0  # Per controllare la frequenza di gestione eventi
    EVENT_RATE = 100  # Gestione eventi ogni 100 ms (10 volte al secondo)
    while True:
        # Aggiungi testo all'immagine
        temp_frame = video_output.copy()
        cv2.putText(
            temp_frame,
            f"Last Speed Limit: {last_speed_limit}",
            (10, 20),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (0, 255, 0),
            2,
            cv2.LINE_AA,
        )
        cv2.putText(
            temp_frame,
            f"Current speed: {speed_car:.0f}",
            (10, 45),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            colorSpeedLimit(speed_car, last_speed_limit),
            2,
            cv2.LINE_AA,
        )

        if red_over:
            val = "Sei passato col rosso"
        else:
            val = ""

        cv2.putText(
            temp_frame,
            val,
            (10, 100),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (0,0,255),
            2,
            cv2.LINE_AA,
        )
            
        cv2.putText(
            temp_frame,
            f"Trafficlight: {traffic_light}",
            (10, 70),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            calcColor(),
            2,
            cv2.LINE_AA,
        )
        # Mostra il frame con il testo
        cv2.imshow('RGB Camera', temp_frame)
        # Interrompi con il tasto 'q'
        if cv2.waitKey(1) == ord('q'):
            break


        current_time = pygame.time.get_ticks()
        if current_time - event_timer > EVENT_RATE:
            event_timer = current_time  # Aggiorna il timer
            # Gestione eventi pygame
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    break
                elif event.type == pygame.KEYUP:  # Rileva il rilascio del tasto
                    if event.key == pygame.K_e:
                        speed_control_activate = not speed_control_activate
                    if event.key == pygame.K_c:
                        vehicle.set_autopilot(True)
                    if event.key == pygame.K_v:
                        vehicle.set_autopilot(False)

            keys = pygame.key.get_pressed()  # Leggi i tasti premuti

            if speed_control_activate and last_speed_limit and keys[pygame.K_w]:
                # Calcolo l'errore rispetto al limite
                error = float(last_speed_limit) - float(speed_car) + 3

                if error < -DEAD_ZONE:  # Sopra il limite di velocità + zona morta
                    throttle = 0
                    brake = max(KP_BRAKE * abs(error), MIN_BRAKE)
                else:  # Sotto o entro il limite + zona morta
                    throttle = max(KP_THROTTLE * error, MIN_THROTTLE)
                    brake = 0
            elif keys[pygame.K_w]:
                # Controllo manuale per accelerare
                throttle = min(throttle + throttle_increment, 1)
                brake = 0
            else:
                throttle = 0
                brake = 0
            
            if keys[pygame.K_s]:  # Controllo manuale per frenare
                brake = min(brake + throttle_increment*4, 1)
                throttle = 0

            if red_over and gear != "Reverse":
                brake = 1
                throttle = 0

            # Controllo sterzo
            if keys[pygame.K_r]:  # Imposta retromarcia
                gear = "Reverse"
                red_over = False
            elif keys[pygame.K_f]:  # Imposta marcia avanti
                gear = "Drive"
            elif keys[pygame.K_a]:  # Sterza a sinistra
                steer = max(steer - steer_increment, -1)
            elif keys[pygame.K_d]:  # Sterza a destra
                steer = min(steer + steer_increment, 1)
            else:
                steer = steer * 0.9  # Ritorno al centro graduale

            # Applicazione dei controlli
            control = carla.VehicleControl()
            control.throttle = throttle
            control.brake = brake
            control.steer = steer
            if gear == "Reverse":
                control.reverse = True
            else:
                control.reverse = False
            vehicle.apply_control(control)

        clock.tick(40)
finally:
    cv2.destroyAllWindows()
    pygame.quit()
    camera.destroy()
    camera_trafficLight.destroy()
    camera_view.destroy()
    vehicle.destroy()
